<a href="https://colab.research.google.com/github/asifsiraz/Beginner-ML-project/blob/main/Mnist_batch_relu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, datasets, utils

# Load MNIST dataset
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()
train_images, test_images = train_images / 255.0, test_images / 255.0

# Define the model with batch normalization and ReLU activation
def create_model():
    model = models.Sequential([
        layers.Flatten(input_shape=(28, 28)),
        layers.Dense(256, activation='relu', kernel_initializer='he_normal'),
        layers.BatchNormalization(),
        layers.Dense(128, activation='relu', kernel_initializer='he_normal'),
        layers.BatchNormalization(),
        layers.Dense(64, activation='relu', kernel_initializer='he_normal'),
        layers.BatchNormalization(),
        layers.Dense(10, activation='softmax')
    ])
    return model

# Train the model
model = create_model()
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=10, batch_size=32, validation_split=0.2)

# Function to calculate the percentage of active nodes in each layer using forward pass
def calculate_active_nodes_percentage(model, x):
    active_nodes_percentages = []
    for layer in model.layers:
        if isinstance(layer, layers.Dense):
            # Perform forward pass
            x = layer(x)
            # Identify active nodes (non-zero activations) for the current layer
            active_nodes = np.sum(x.numpy() != 0)
            total_nodes = np.prod(x.shape[-1])
            active_nodes_percentage = (active_nodes / total_nodes) * 100
            active_nodes_percentages.append(active_nodes_percentage)
    return active_nodes_percentages

# Create a sample batch for the forward pass
# Flatten the input data before passing it to the model
sample_batch_flattened = sample_batch.reshape(sample_batch.shape[0], -1)

# Calculate the percentage of active nodes for each layer using forward pass
active_nodes_percentages = calculate_active_nodes_percentage(model, sample_batch_flattened)
for i, percentage in enumerate(active_nodes_percentages, start=1):
    print(f"Layer {i}: {percentage:.2f}% active nodes")

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print("Test accuracy:", test_accuracy)


Epoch 1/10
1500/1500 [==============================] - 13s 8ms/step - loss: 0.2614 - accuracy: 0.9215 - val_loss: 0.1167 - val_accuracy: 0.9629
Epoch 2/10
1500/1500 [==============================] - 11s 8ms/step - loss: 0.1270 - accuracy: 0.9609 - val_loss: 0.0970 - val_accuracy: 0.9703
Epoch 3/10
1500/1500 [==============================] - 11s 8ms/step - loss: 0.0956 - accuracy: 0.9704 - val_loss: 0.1005 - val_accuracy: 0.9701
Epoch 4/10
1500/1500 [==============================] - 11s 8ms/step - loss: 0.0788 - accuracy: 0.9747 - val_loss: 0.0851 - val_accuracy: 0.9752
Epoch 5/10
1500/1500 [==============================] - 11s 7ms/step - loss: 0.0678 - accuracy: 0.9786 - val_loss: 0.0914 - val_accuracy: 0.9732
Epoch 6/10
1500/1500 [==============================] - 12s 8ms/step - loss: 0.0599 - accuracy: 0.9806 - val_loss: 0.0850 - val_accuracy: 0.9766
Epoch 7/10
1500/1500 [==============================] - 12s 8ms/step - loss: 0.0520 - accuracy: 0.9826 - val_loss: 0.0835 - val_ac

In [8]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, datasets, utils
from tensorflow.keras.regularizers import l1  # Import the L1 regularization

# Load MNIST dataset
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()
train_images, test_images = train_images / 255.0, test_images / 255.0

# Define the model with batch normalization and ReLU activation, and L1 regularization
def create_model_with_l1():
    model = models.Sequential([
        layers.Flatten(input_shape=(28, 28)),
        layers.Dense(256, activation='relu', kernel_initializer='he_normal', kernel_regularizer=l1(0.01)),
        layers.BatchNormalization(),
        layers.Dense(128, activation='relu', kernel_initializer='he_normal', kernel_regularizer=l1(0.01)),
        layers.BatchNormalization(),
        layers.Dense(64, activation='relu', kernel_initializer='he_normal', kernel_regularizer=l1(0.01)),
        layers.BatchNormalization(),
        layers.Dense(10, activation='softmax')
    ])
    return model

learning_rate = 0.001


# Train the model with L1 regularization
model_with_l1 = create_model_with_l1()
model_with_l1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_with_l1.fit(train_images, train_labels, epochs=10, batch_size=32, validation_split=0.2)

# Function to calculate the percentage of active nodes in each layer using forward pass
def calculate_active_nodes_percentage(model, x):
    active_nodes_percentages = []
    for layer in model.layers:
        if isinstance(layer, layers.Dense):
            # Perform forward pass
            x = layer(x)
            # Identify active nodes (non-zero activations) for the current layer
            active_nodes = np.sum(x.numpy() != 0)
            total_nodes = np.prod(x.shape[-1])
            active_nodes_percentage = (active_nodes / total_nodes) * 100
            active_nodes_percentages.append(active_nodes_percentage)
    return active_nodes_percentages

# Create a sample batch for the forward pass
sample_batch_flattened = train_images[:32].reshape(32, -1)

# Calculate the percentage of active nodes for each layer using forward pass
active_nodes_percentages = calculate_active_nodes_percentage(model_with_l1, sample_batch_flattened)
for i, percentage in enumerate(active_nodes_percentages, start=1):
    print(f"Layer {i}: {percentage:.2f}% active nodes")

# Evaluate the model on the test set
test_loss, test_accuracy = model_with_l1.evaluate(test_images, test_labels)
print("Test accuracy:", test_accuracy)

def calculate_sparsity(model):
    total_params = 0
    zero_params = 0
    for layer in model.layers:
        if isinstance(layer, layers.Dense):
            # Get the layer's weights
            weights = layer.get_weights()[0]
            total_params += np.prod(weights.shape)
            zero_params += np.sum(np.abs(weights) < 1e-5)  # Count the number of near-zero weights
    sparsity = zero_params / total_params * 100
    return sparsity

# Calculate the sparsity of the model
sparsity = calculate_sparsity(model_with_l1)
print(f"Model Sparsity: {sparsity:.2f}%")


Epoch 1/10
1500/1500 [==============================] - 14s 8ms/step - loss: 7.4583 - accuracy: 0.8338 - val_loss: 1.6411 - val_accuracy: 0.8356
Epoch 2/10
1500/1500 [==============================] - 12s 8ms/step - loss: 1.3477 - accuracy: 0.8818 - val_loss: 1.3031 - val_accuracy: 0.8713
Epoch 3/10
1500/1500 [==============================] - 12s 8ms/step - loss: 1.1979 - accuracy: 0.8905 - val_loss: 2.1923 - val_accuracy: 0.6378
Epoch 4/10
1500/1500 [==============================] - 12s 8ms/step - loss: 1.1348 - accuracy: 0.8966 - val_loss: 1.2306 - val_accuracy: 0.8720
Epoch 5/10
1500/1500 [==============================] - 12s 8ms/step - loss: 1.1138 - accuracy: 0.8968 - val_loss: 1.1136 - val_accuracy: 0.8847
Epoch 6/10
1500/1500 [==============================] - 11s 7ms/step - loss: 1.0843 - accuracy: 0.8989 - val_loss: 1.0817 - val_accuracy: 0.8951
Epoch 7/10
1500/1500 [==============================] - 11s 7ms/step - loss: 1.0660 - accuracy: 0.8978 - val_loss: 1.1126 - val_ac

In [6]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, datasets, utils
from tensorflow.keras.regularizers import l1  # Import the L1 regularization

# Load MNIST dataset
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()
train_images, test_images = train_images / 255.0, test_images / 255.0

# Define the model with batch normalization and ReLU activation, and L1 regularization
def create_model_with_l1():
    model = models.Sequential([
        layers.Flatten(input_shape=(28, 28)),
        layers.Dense(256, activation='relu', kernel_initializer='he_normal', kernel_regularizer=l1(0.001)),
        layers.BatchNormalization(),
        layers.Dense(128, activation='relu', kernel_initializer='he_normal', kernel_regularizer=l1(0.001)),
        layers.BatchNormalization(),
        layers.Dense(64, activation='relu', kernel_initializer='he_normal', kernel_regularizer=l1(0.001)),
        layers.BatchNormalization(),
        layers.Dense(10, activation='softmax')
    ])
    return model

learning_rate = 0.001


# Train the model with L1 regularization
model_with_l1 = create_model_with_l1()
model_with_l1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_with_l1.fit(train_images, train_labels, epochs=10, batch_size=64, validation_split=0.2)

# Function to calculate the percentage of active nodes in each layer using forward pass
def calculate_active_nodes_percentage(model, x):
    active_nodes_percentages = []
    for layer in model.layers:
        if isinstance(layer, layers.Dense):
            # Perform forward pass
            x = layer(x)
            # Identify active nodes (non-zero activations) for the current layer
            active_nodes = np.sum(x.numpy() != 0)
            total_nodes = np.prod(x.shape[-1])
            active_nodes_percentage = (active_nodes / total_nodes) * 100
            active_nodes_percentages.append(active_nodes_percentage)
    return active_nodes_percentages

# Create a sample batch for the forward pass
sample_batch_flattened = train_images[:64].reshape(64, -1)

# Calculate the percentage of active nodes for each layer using forward pass
active_nodes_percentages = calculate_active_nodes_percentage(model_with_l1, sample_batch_flattened)
for i, percentage in enumerate(active_nodes_percentages, start=1):
    print(f"Layer {i}: {percentage:.2f}% active nodes")

# Evaluate the model on the test set
test_loss, test_accuracy = model_with_l1.evaluate(test_images, test_labels)
print("Test accuracy:", test_accuracy)


Epoch 1/10
750/750 [==============================] - 10s 11ms/step - loss: 3.7749 - accuracy: 0.9093 - val_loss: 1.5384 - val_accuracy: 0.9222
Epoch 2/10
750/750 [==============================] - 7s 9ms/step - loss: 1.1372 - accuracy: 0.9292 - val_loss: 0.9673 - val_accuracy: 0.9106
Epoch 3/10
750/750 [==============================] - 9s 11ms/step - loss: 0.7468 - accuracy: 0.9398 - val_loss: 0.6338 - val_accuracy: 0.9467
Epoch 4/10
750/750 [==============================] - 8s 11ms/step - loss: 0.5899 - accuracy: 0.9468 - val_loss: 0.6392 - val_accuracy: 0.9214
Epoch 5/10
750/750 [==============================] - 7s 9ms/step - loss: 0.5160 - accuracy: 0.9505 - val_loss: 0.5632 - val_accuracy: 0.9345
Epoch 6/10
750/750 [==============================] - 8s 11ms/step - loss: 0.4855 - accuracy: 0.9514 - val_loss: 0.4800 - val_accuracy: 0.9493
Epoch 7/10
750/750 [==============================] - 7s 10ms/step - loss: 0.4561 - accuracy: 0.9541 - val_loss: 0.5690 - val_accuracy: 0.9212


In [7]:
def calculate_sparsity(model):
    total_params = 0
    zero_params = 0
    for layer in model.layers:
        if isinstance(layer, layers.Dense):
            # Get the layer's weights
            weights = layer.get_weights()[0]
            total_params += np.prod(weights.shape)
            zero_params += np.sum(np.abs(weights) < 1e-5)  # Count the number of near-zero weights
    sparsity = zero_params / total_params * 100
    return sparsity

# Calculate the sparsity of the model
sparsity = calculate_sparsity(model_with_l1)
print(f"Model Sparsity: {sparsity:.2f}%")

Model Sparsity: 6.37%
